In [1]:
# Naver finance_MariaDB
from datetime import datetime
import backtrader as bt
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
%matplotlib inline
plt.rcParams['font.family'] = 'Malgun Gothic' 
plt.rcParams['axes.unicode_minus'] = False

investar_parent_path = 'C:\\myPackage'
if investar_parent_path not in sys.path:
    sys.path.append(investar_parent_path)
from Investar.Analyzer import MarketDB
mk = MarketDB()

company = '207940'
start_date = "2025-01-01"

# 1) Strategy
class MyStrategy(bt.Strategy):
    params = (('rsi_period', 21), ('oversold', 30), ('overbought', 70),)

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=self.p.rsi_period)

    def notify_order(self, order):  # ①
        if order.status in [order.Submitted, order.Accepted]:
            return
        if order.status in [order.Completed]:  # ② 
            if order.isbuy():
                self.log(f'BUY  : 주가 {order.executed.price:,.0f}, '
                    f'수량 {order.executed.size:,.0f}, '
                    f'수수료 {order.executed.comm:,.0f}, '
                    f'자산 {cerebro.broker.getvalue():,.0f}') 
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log(f'SELL : 주가 {order.executed.price:,.0f}, '
                    f'수량 {order.executed.size:,.0f}, '
                    f'수수료 {order.executed.comm:,.0f}, '
                    f'자산 {cerebro.broker.getvalue():,.0f}') 
            self.bar_executed = len(self)
        elif order.status in [order.Canceled]:
            self.log('ORDER CANCELD')
        elif order.status in [order.Margin]:
            self.log('ORDER MARGIN')
        elif order.status in [order.Rejected]:
            self.log('ORDER REJECTED')
        self.order = None
   
    def next(self):
        if not self.position:
            if self.rsi < self.p.oversold:
                self.order = self.buy()
        else:
            if self.rsi > self.p.overbought:
                self.order = self.sell()
                
    def log(self, txt, dt=None):
        dt = self.datas[0].datetime.date(0)
        print(f'[{dt.isoformat()}] {txt}')

# 2) Download data using MariaDB
#df = yf.download(ticker, start = start_date,  auto_adjust=True)
#df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]
df = mk.get_daily_price(company, start_date)
# 컬럼 Rename (반드시 필요!)
df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})
# date 컬럼이 있을 경우 인덱스로 지정
if 'date' in df.columns:
    df.index = pd.to_datetime(df['date'])
print(df)

# 3) Set up Backtrader
data = bt.feeds.PandasData(dataname=df)

cerebro = bt.Cerebro()
cerebro.addstrategy(MyStrategy)

cerebro.adddata(data)
cerebro.broker.setcash(10000000)
cerebro.broker.setcommission(commission=0.0014)
cerebro.addsizer(bt.sizers.PercentSizer, percents=90)

# 4) Run backtest
print(f"Initial Portfolio Value : {cerebro.broker.getvalue():,.0f} KRW")
cerebro.run()
print(f"Final Portfolio Value   : {cerebro.broker.getvalue():,.0f} KRW")

# 5) Plot (avoid Javascript error)
cerebro.plot(iplot=False, style='candlestick', figsize=(20, 15)) 
#Backtrader’s plotting uses matplotlib + some JavaScript that Jupyter does not load correctly
#IDLE 실행하거나, %matplotlib inline 추가

end_date is initialized to '2025-11-30'
              code        date     Open     High      Low    Close    diff  \
date                                                                         
2025-01-02  207940  2025-01-02   950000   952000   924000   934000   15000   
2025-01-03  207940  2025-01-03   934000   951000   932000   940000    6000   
2025-01-06  207940  2025-01-06   940000   948000   932000   946000    6000   
2025-01-07  207940  2025-01-07   951000  1005000   950000   983000   37000   
2025-01-08  207940  2025-01-08   991000  1011000   989000  1008000   25000   
...            ...         ...      ...      ...      ...      ...     ...   
2025-11-24  207940  2025-11-24  1797000  1841000  1650000  1789000    8000   
2025-11-25  207940  2025-11-25  1812000  1812000  1586000  1627000  162000   
2025-11-26  207940  2025-11-26  1640000  1681000  1600000  1651000   24000   
2025-11-27  207940  2025-11-27  1662000  1687000  1640000  1646000    5000   
2025-11-28  207940  2025

[[<Figure size 640x480 with 5 Axes>]]